In [30]:
import os
import pickle
import binascii

def crc32_checksum(filename):
    buf = open(filename,'rb').read()
    buf = (binascii.crc32(buf) & 0xFFFFFFFF)
    return "%08X" % buf

def get_kcs(path_192) :
    crc32 = str(crc32_checksum(path_192))
    size = str(os.path.getsize(path_192))
    f = os.path.split(path_192)[1]
    key = f[:8]
    kcs = key + crc32 + size 
    return kcs

def read_cks(path_kcs) :
    with open(path_kcs, "rb") as pkl :
        return pickle.load(pkl)

def delete_kcs(filePath, dict_kcs) :
    k = os.path.split(filePath)[1][:8]
    c = str(crc32_checksum(filePath))
    s = str(os.path.getsize(filePath))
    kcs = k + c + s
    if kcs in dict_kcs :
        del dict_kcs[kcs]
        print("삭제하였습니다.")
    else :
      print(filePath, kcs, "없는 키값")

def write_cks(path_kcs, dict_kcs) :
    with open(path_kcs, "wb") as pkl :
        pickle.dump(dict_kcs, pkl)

path_kcs = r"\\192.168.0.75\스캔파일\스캔파일log\_project\파일\중복조사\kcs별 파일정보_솔림헬프.pkl"

In [ ]:
# 읽기
dict_kcs = read_cks(path_kcs)
print("최초dict_kcs개수 :", len(dict_kcs))

In [ ]:
# 삭제하기
####################################
path_192 = r"\\192.168.0.75\스캔파일\새 스캔파일(업로드)\구폴더작업\20408432_정준영_기타_사해행위품의서.pdf"
####################################
delete_kcs(path_192, dict_kcs)

In [ ]:
# 쓰기
write_cks(path_kcs, dict_kcs)